## Web traffic
Using the information from Jeremy Howards's data. https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [1]:
# How many rows should we have remaining
# 0 means everyone
tableNumMod= 0

# How many columns should be here
#Array size is 794 : 794 ran out of data
#398 will half it
#700 will bring it to 90 days
#600 does not run out of memory unless submissions
#500?
tableColMod= 470

# Are we filling the submission data?
SubmissionRun = True

# Saves Weights and loads as needed
WantToSave="No"
WantToLoad="No"

## Setup

In [2]:
#import notebook_util
#notebook_util.setup_one_gpu()

In [3]:
# I like the dings! You can comment this out
get_ipython().magic('load_ext cellevents')

In [4]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
    #display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
    display(Audio(url='http://starmen.net/mother2/soundfx/eb_win.wav', autoplay=True))
    
def RunningEpochs():
    #display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
    display(Audio(url='http://starmen.net/mother2/soundfx/enterbattle.wav', autoplay=True))
## Insert whatever audio file you want above


time: 3.77 ms


In [5]:
%matplotlib inline

time: 170 ms


In [6]:
# All of the dependencies
import math, keras, datetime, pandas as pd, numpy as np, keras.backend as K
import matplotlib.pyplot as plt, xgboost, operator, random, pickle
import h5py
from keras.models import load_model
import pandas as pd
from datetime import datetime
import re
from tensorflow.python.client import device_lib
from utils2 import *
# Breaks out dates into good data
from isoweek import Week
from pandas_summary import DataFrameSummary
# Options determine the way floating point numbers, arrays and other NumPy objects are displayed
np.set_printoptions(threshold=20, edgeitems=10)
from IPython.display import HTML
from datetime import datetime
date = datetime.strptime("2015-07-01", "%Y-%m-%d")  

Using TensorFlow backend.
/home/jd/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


time: 2.26 s


In [7]:
# Limits the amount of GPU is used up
#limit_mem()
import tensorflow as tf

time: 3.49 ms


In [8]:
# lists out the GPUs
print (device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16938822025251334683
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 7938919629
locality {
  bus_id: 1
}
incarnation: 13473741503910362592
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0"
, name: "/gpu:1"
device_type: "GPU"
memory_limit: 7950188544
locality {
  bus_id: 1
}
incarnation: 5745575823229464886
physical_device_desc: "device: 1, name: GeForce GTX 1080, pci bus id: 0000:06:00.0"
]
time: 292 ms


In [9]:
# Set up pathing
path = '/home/jd/data/webTraffic/'

time: 592 µs


### Functions

In [10]:
# Adding datepart
# Used in data Cleaning
def add_datepart(df):
    df.Date = pd.to_datetime(df.Date)
    df["Year"] = df.Date.dt.year
    df["Month"] = df.Date.dt.month
    df["Week"] = df.Date.dt.week
    df["Day"] = df.Date.dt.day
    df["DayOfWeek"] = df.Date.dt.weekday

time: 3.8 ms


In [11]:
# this did not work as intended
def add_pagebreak(df):
    pd.options.mode.chained_assignment = None
    df['frontname']= df['Page'].str.partition('.')[0]
    df['name'] = df['frontname'].str.rpartition('_')[0]
    df['locale'] = df['frontname'].str.rpartition('_')[2]
    df['backname'] = df['Page'].str.partition('.')[2]
    df['article'] = df['backname'].str.partition('_')[0]
    df['access'] = df['backname'].str.partition('_')[2]
    df['agent'] = df['backname'].str.rpartition('_')[2]

time: 6.43 ms


In [12]:
# Is used to merge the pandas together
# Used in Data Cleaning
def join_df(left, right, left_on, right_on=None):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", "_y"))

time: 1.95 ms


In [13]:
# Basic class
class elapsed(object):
    def __init__(self, fld):
        self.fld = fld
        self.last = pd.to_datetime(np.nan)
        self.last_store = 0
        
    def get(self, row):
        if row.Store != self.last_store:
            self.last = pd.to_datetime(np.nan)
            self.last_store = row.Store
        if (row[self.fld]): self.last = row.Date
        return row.Date-self.last

time: 6.39 ms


In [14]:
def add_elapsed(fld, prefix):
    sh_el = elapsed(fld)
    df[prefix+fld] = df.apply(sh_el.get, axis=1)

time: 1.97 ms


In [15]:
# Makes life very easy.. learn more about it
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler

time: 6.5 ms


In [16]:
#Used in validation
def cat_preproc(dat):
    return cat_map_fit.transform(dat).astype(np.int64)

time: 916 µs


In [17]:
#Used in validation
def contin_preproc(dat):
    return contin_map_fit.transform(dat).astype(np.float32)

time: 1.15 ms


In [18]:
# Will need to switch this out
# Used in modeling
from numba import jit
import math

def rmspe(y_pred, targ ):# target = y_valid_orig
    pct_var = (targ - y_pred)/targ
    return math.sqrt(np.square(pct_var).mean())

time: 176 ms


In [19]:
# Used at the end of models 
def log_max_inv(preds, mx ): #mx= max_log_y
    return np.exp(preds * mx)

time: 1.04 ms


In [20]:
# Used at the end of models 
def normalize_inv(preds):
    return preds * ystd + ymean

time: 926 µs


In [21]:
# Used for making the models
def split_cols(arr): return np.hsplit(arr,arr.shape[1])

time: 1.05 ms


In [22]:
# Used for making the models
def cat_map_info(feat): return feat[0], len(feat[1].classes_)

time: 826 µs


In [23]:
# Used for making the models
def my_init(scale):
    return lambda shape, name=None: initializations.uniform(shape, scale=scale, name=name)

time: 1.01 ms


In [24]:
# Used for making the models
def emb_init(shape, name=None): 
    return initializations.uniform(shape, scale=2/(shape[1]+1), name=name)

time: 1.33 ms


In [25]:
# Used for making the models
def get_contin(feat):
    name = feat[0][0]
    inp = Input((1,), name=name+'_in')
    return inp, Dense(1, name=name+'_d', init=my_init(1.))(inp)

time: 2.01 ms


In [26]:
# Used for making the models
def get_emb(feat):
    name, c = cat_map_info(feat)
    #c2 = cat_var_dict[name]
    c2 = (c+1)//2
    if c2>50: c2=50
    inp = Input((1,), dtype='int64', name=name+'_in')
    # , W_regularizer=l2(1e-6)
    u = Flatten(name=name+'_flt')(Embedding(c, c2, input_length=1, init=emb_init)(inp))
#     u = Flatten(name=name+'_flt')(Embedding(c, c2, input_length=1)(inp))
    return inp,u

time: 6.31 ms


## Load Data Tables

In [27]:
# Pulls in the data
joined = pd.read_csv(path+'train_2'+'.csv', low_memory=False)

time: 25.1 s


In [28]:
#Checks it length is correct
len(joined)

145063

time: 2.48 ms


In [29]:
#Shows the first bit
display(joined.head(2))

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-09-01,2017-09-02,2017-09-03,2017-09-04,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,19.0,33.0,33.0,18.0,16.0,27.0,29.0,23.0,54.0,38.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,32.0,30.0,11.0,19.0,54.0,25.0,26.0,23.0,13.0,81.0


time: 18.3 ms


# Select the size of the array

In [30]:
# Modifies the joined rows and columns
if (tableNumMod>0):
    joined=joined[:tableNumMod]

if (tableColMod>0):
    joined=joined.drop(joined.columns[[range(1,(794-tableColMod),1)]], axis=1)

time: 91.7 ms


In [31]:
# Causes an exploding 0
joined=joined.replace(0, .00000001)

time: 395 ms


In [32]:
# Its tough deciding to clip or not.
#Needs clipping for smape
joined=joined.fillna(1e-8)
#tables=tables.fillna(0)

time: 231 ms


In [33]:
# Breaks this into a more useable graph 
joined=pd.melt(joined, id_vars="Page", var_name="Date", value_name="Visits")

time: 3.08 s


In [34]:
#Adds the dates on
add_datepart(joined)

time: 26.9 s


In [35]:
# Loads the indivators that are split out columns for the datea
pagesplit = pd.read_csv(path+'betterdatacheck2'+'.csv', low_memory=False)
pagesplit=pagesplit.drop('Unnamed: 0', axis=1)

time: 253 ms


In [36]:
# merges it with Page
joined=join_df(joined,pagesplit, "Page", "Page")

time: 23.7 s


In [37]:
# Full set is not needed for 13 Mins; seems too long
#tables.to_csv(path+'delete.csv')

time: 558 µs


In [38]:
joined.sort_values(['Date'])

,Page,Date,Visits,Year,Month,Week,Day,DayOfWeek,name,locale,article,access,agent
0,2NE1_zh.wikipedia.org_all-access_spider,2016-05-09,2.200000e+01,2016,5,19,9,0,2NE1,zh,wikipedia,all-access,spider
96717,Elemento_químico_es.wikipedia.org_mobile-web_a...,2016-05-09,2.031000e+03,2016,5,19,9,0,Elemento_químico,es,wikipedia,mobile-web,all-agents
96716,Paleolítico_es.wikipedia.org_mobile-web_all-ag...,2016-05-09,1.422000e+03,2016,5,19,9,0,Paleolítico,es,wikipedia,mobile-web,all-agents
96715,América_de_Cali_es.wikipedia.org_mobile-web_al...,2016-05-09,1.400000e+01,2016,5,19,9,0,América_de_Cali,es,wikipedia,mobile-web,all-agents
96714,Historia_del_fútbol_es.wikipedia.org_mobile-we...,2016-05-09,2.295000e+03,2016,5,19,9,0,Historia_del_fútbol,es,wikipedia,mobile-web,all-agents
96713,Ángel_Di_María_es.wikipedia.org_mobile-web_all...,2016-05-09,2.940000e+02,2016,5,19,9,0,Ángel_Di_María,es,wikipedia,mobile-web,all-agents
96712,Coordenadas_geográficas_es.wikipedia.org_mobil...,2016-05-09,1.527000e+03,2016,5,19,9,0,Coordenadas_geográficas,es,wikipedia,mobile-web,all-agents
96711,Rafael_Leónidas_Trujillo_es.wikipedia.org_mobi...,2016-05-09,1.672000e+03,2016,5,19,9,0,Rafael_Leónidas_Trujillo,es,wikipedia,mobile-web,all-agents
96710,Andrés_Iniesta_es.wikipedia.org_mobile-web_all...,2016-05-09,5.610000e+02,2016,5,19,9,0,Andrés_Iniesta,es,wikipedia,mobile-web,all-agents
96709,Tarzán_es.wikipedia.org_mobile-web_all-agents,2016-05-09,2.850000e+02,2016,5,19,9,0,Tarzán,es,wikipedia,mobile-web,all-agents


time: 14.4 s


## Load the data that we are asked to predict
This is probably overly complex but I setup the predict section together to merge with train data

In [39]:
# Grabs the submission template and all the keys
submission = pd.read_csv(path+'sample_submission_2.csv')
# FIrst round
#key = pd.read_csv(path+'key_2.csv')
# use modified key RevisedKey_2
key = pd.read_csv(path+'RevisedKey_2.csv')

time: 14.6 s


In [41]:
# Joins the dateframes together
subm=join_df(submission,key, "Id", "Id")

time: 6.51 s


In [42]:
# Its ugly but it will work.
# Pulls apart all of the valuable information and joins it together
def submission_builder(df):
    #df["Date"] = df['Page'].str.extract('(201\d-\d\d-\d\d)', expand=True)
    #df["Date"] = df['Page'].str.extract('(201[567]-\d\d-\d\d)', expand=True)
    
    # Junk came alive at 12 hours before the competition ended. Another date was being captured.
    # I couldn't regex it out
    df["Junk"] = df['Page'].str.extract('([sr]_201[567]-\d\d-\d\d)', expand=True)
    # Pulls the real date out of the junk.
    df["Date"] = df['Junk'].str.extract('(201[567]-\d\d-\d\d)', expand=True)
    df["Name"] = df['Page'].str.rsplit('_', expand=True, n=1)[0]
    df.Date = pd.to_datetime(df.Date)
    df["Year"] = df.Date.dt.year
    df["Month"] = df.Date.dt.month
    df["Week"] = df.Date.dt.week
    df["Day"] = df.Date.dt.day
    df["DayOfWeek"] = df.Date.dt.weekday
    # I want to know if things are being predicted or not; -1 needs a Visit
    df["Visits"] = -1

time: 10.1 ms


In [43]:
# last minute date crisis
# so i needed to do the same thing without modifying 
def submission_builder2(df):
    #df["Date"] = df['Page'].str.extract('(201\d-\d\d-\d\d)', expand=True)
    #df["Date"] = df['Page'].str.extract('(201[567]-\d\d-\d\d)', expand=True)
    df["Junk"] = df['Page'].str.extract('([sr]_201[567]-\d\d-\d\d)', expand=True)
    df["Date"] = df['Junk'].str.extract('(201[567]-\d\d-\d\d)', expand=True)
    df["Name"] = df['Page'].str.rsplit('_', expand=True, n=1)[0]
    df.Date = pd.to_datetime(df.Date)
    df["Year"] = df.Date.dt.year
    df["Month"] = df.Date.dt.month
    df["Week"] = df.Date.dt.week
    df["Day"] = df.Date.dt.day
    df["DayOfWeek"] = df.Date.dt.weekday

time: 7.83 ms


In [44]:
# Now lets actually run the split
submission_builder(subm)

time: 1min 1s


In [47]:
#Join the extra data into the prediction model
subm=join_df(subm,pagesplit, "Name", "Page")

time: 3.79 s


In [48]:
# Needed something to place the ID back into the 
columnsTitles=["Id","Page","Name","Date","Visits"]
ThebestestKey=subm.reindex(columns=columnsTitles)

time: 2.7 s


In [49]:
ThebestestKey

,Id,Page,Name,Date,Visits
0,0b293039387a,007_スペクター_ja.wikipedia.org_all-access_all-agen...,007_スペクター_ja.wikipedia.org_all-access_all-agents,2017-09-13,-1
1,7114389dd824,007_スペクター_ja.wikipedia.org_all-access_all-agen...,007_スペクター_ja.wikipedia.org_all-access_all-agents,2017-09-14,-1
2,057b02ff1f09,007_スペクター_ja.wikipedia.org_all-access_all-agen...,007_スペクター_ja.wikipedia.org_all-access_all-agents,2017-09-15,-1
3,bd2aca21caa3,007_スペクター_ja.wikipedia.org_all-access_all-agen...,007_スペクター_ja.wikipedia.org_all-access_all-agents,2017-09-16,-1
4,c0effb42cdd5,007_スペクター_ja.wikipedia.org_all-access_all-agen...,007_スペクター_ja.wikipedia.org_all-access_all-agents,2017-09-17,-1
5,4ccd369adefc,007_スペクター_ja.wikipedia.org_all-access_all-agen...,007_スペクター_ja.wikipedia.org_all-access_all-agents,2017-09-18,-1
6,67d5c2ebc330,007_スペクター_ja.wikipedia.org_all-access_all-agen...,007_スペクター_ja.wikipedia.org_all-access_all-agents,2017-09-19,-1
7,1c51606e416b,007_スペクター_ja.wikipedia.org_all-access_all-agen...,007_スペクター_ja.wikipedia.org_all-access_all-agents,2017-09-20,-1
8,0f27b91481db,007_スペクター_ja.wikipedia.org_all-access_all-agen...,007_スペクター_ja.wikipedia.org_all-access_all-agents,2017-09-21,-1
9,07f5f08a49fa,007_スペクター_ja.wikipedia.org_all-access_all-agen...,007_スペクター_ja.wikipedia.org_all-access_all-agents,2017-09-22,-1


time: 16.4 ms


In [50]:
# get it saved for the other program
pickle.dump(ThebestestKey, open('ThebestestKey.pickle', 'wb'))

time: 11 s


In [51]:
# the prediction tables have all of this good information on it... only this though... drop everything else.
columnsTitles=["Page","Name","Date","Visits", "Year", "Month", "Week","Day","DayOfWeek", "name", "locale","article","access","agent"]
ptables=subm.reindex(columns=columnsTitles)
ptables[:1]

,Page,Name,Date,Visits,Year,Month,Week,Day,DayOfWeek,name,locale,article,access,agent
0,007_スペクター_ja.wikipedia.org_all-access_all-agen...,007_スペクター_ja.wikipedia.org_all-access_all-agents,2017-09-13,-1,2017,9,37,13,2,007_スペクター,ja,wikipedia,all-access,all-agents


time: 541 ms


In [52]:
# and i dont need Page but name... so lets drop it
ptables=ptables.drop('Page', axis=1)
#overly complicated way to rename Page
ptables.columns = ["Page","Date","Visits", "Year", "Month", "Week","Day","DayOfWeek", "name", "locale","article","access","agent"]

time: 388 ms


In [53]:
# Here is the prediction tables all done
ptables[:1]

,Page,Date,Visits,Year,Month,Week,Day,DayOfWeek,name,locale,article,access,agent
0,007_スペクター_ja.wikipedia.org_all-access_all-agents,2017-09-13,-1,2017,9,37,13,2,007_スペクター,ja,wikipedia,all-access,all-agents


time: 8.11 ms


## Start Joining all of the data together

In [54]:
# I want the length before I add in 
atspace=len(joined)

time: 1.03 ms


In [57]:
# Appends the prediction everything to tables; This was easier to make sure everything that happened to one
# happened to the other
pickle.dump(joined, open('joined.pickle', 'wb'))
if (SubmissionRun):
    joined=joined.append(ptables)

time: 1min 43s


In [60]:
# Sort everything by date
joined["Date"] = pd.to_datetime(joined.Date)
# what are my columns
joined.columns

Index(['Page', 'Date', 'Visits', 'Year', 'Month', 'Week', 'Day', 'DayOfWeek',
       'name', 'locale', 'article', 'access', 'agent'],
      dtype='object')

time: 255 ms


In [62]:
# the dictionary of cateogry variables; weights are placed for reasons
# These are the variables I felt were important. I added more weight depending on how important they were 
# for correlation. I don't understand it too much.
# Page	Date	Visits	Year	Month	Week	Day	DayOfWeek	name	locale	article	access	agent
cat_var_dict = {'Year' : 1, 'Month' :6, 
                'Week': 2, 'Day': 3, 'DayOfWeek':10,'locale':6,'agent':7, 'article':8, 'name':30,
                'access':9}
# These values should have no bearing on the model
#'Visits': 15,'Page': 20,'Date': 10,

time: 11.7 ms


In [63]:
# Don't understand this one yet
cat_vars = [o[0] for o in 
            sorted(cat_var_dict.items(), key=operator.itemgetter(1), reverse=True)]

time: 15.2 ms


In [64]:
# I dont really have values that are much different
contin_vars = {'Year'}
#contin_vars = {'Year','Date','Visits'}

time: 9.69 ms


In [65]:
for v in contin_vars: joined.loc[joined[v].isnull(), v] = 0
#for v in cat_vars: joined.loc[joined[v].isnull(), v] = ""


time: 22.5 s


In [66]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

time: 12 ms


In [67]:
# maps all the important variables out
cat_mapper = DataFrameMapper(cat_maps)
cat_map_fit = cat_mapper.fit(joined)
cat_cols = len(cat_map_fit.features)
cat_cols

10

time: 5min 10s


In [68]:
# maps all contin out
contin_mapper = DataFrameMapper(contin_maps)
contin_map_fit = contin_mapper.fit(joined)
contin_cols = len(contin_map_fit.features)
contin_cols

/home/jd/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


1

time: 622 ms


In [69]:
# We start saving these things that took a long time to train
pickle.dump(contin_map_fit, open('contin_maps.pickle', 'wb'))
pickle.dump(cat_map_fit, open('cat_maps.pickle', 'wb'))

time: 65.3 ms


In [70]:
[len(o[1].classes_) for o in cat_map_fit.features]

[49176, 7, 3, 4, 2, 10, 12, 31, 52, 2]

time: 2.04 ms


## Sample Data

In [71]:
#joined = joined[joined.Visits!=0]
n = len(joined)

time: 5.6 ms


In [72]:
samp_size = 100000
if (samp_size>n):
    samp_size = n
np.random.seed(42)
idxs = sorted(np.random.choice(n, samp_size, replace=False))

time: 3.25 s


In [73]:
joined_samp = joined.iloc[idxs].set_index("Date")
joined_samp = joined.iloc[idxs].set_index("Page")

time: 352 ms


In [74]:
samp_size = n
joined_samp = joined.set_index("Date")
#joined_samp = joined.set_index("Page")

time: 4.08 s


### Save values for validation

In [76]:
# Actual Splitting of the data
# joined_train = our training data
# joined_valid = our validation 
# joined_predict= the values we will be predicting; 

# If its a submission run we will use the prediction data. Otherwise we will make the prediction value 
if (SubmissionRun):
    # Breaks the training size into the valid and train sets
    # atspace
    train_ratio = 0.8
    train_size = int(atspace * train_ratio)
    ##
    joined_train = joined_samp[:train_size]
    joined_valid = joined_samp[train_size:atspace]
    joined_predict= joined_samp[atspace:]
else:
    # if its not a submission I have to spend more time cutting up more of the valid data
    # Otherwise the same thing
    if (tableNumMod==0):
        tableNumMod= 145063
    train_size= len(joined_samp)-tableNumMod*120
    train_size1= len(joined_samp)-tableNumMod*60
    print (train_size,train_size1)
    joined_train = joined_samp[:train_size]
    joined_valid = joined_samp[train_size:train_size1]
    # in hindsight I should call this test predictions. 
    joined_predict= joined_samp[train_size1:]
len(joined_train),len(joined_valid),  len(joined_predict)

(56864696, 14216174, 8993906)

time: 22.6 ms


In [78]:
# Resets the index for the data set; and ensures there is one
joined_train=joined_train.reset_index(level=None)
joined_valid=joined_valid.reset_index(level=None)
joined_predict=joined_predict.reset_index(level=None)

time: 26.9 s


In [79]:
# 9 min
# I do not understand what is being output
cat_map_train = cat_preproc(joined_train)

time: 4min 58s


In [80]:
#strips off the headers to prepare for data
cat_map_valid = cat_preproc(joined_valid)

time: 1min 8s


In [81]:
#completes preporcessing
contin_map_train = contin_preproc(joined_train)
contin_map_valid = contin_preproc(joined_valid)
contin_map_predict = contin_preproc(joined_predict)

/home/jd/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


time: 716 ms


In [82]:
#completes preporcessing; dont need to do it for predict.
cat_map_train =cat_preproc(joined_train)
cat_map_valid =cat_preproc(joined_valid)

MemoryError: 

time: 4min 47s


In [ ]:
cat_map_predict = cat_preproc(joined_predict)

In [ ]:
y_train_orig = joined_train.Visits
y_valid_orig = joined_valid.Visits
y_predict_orig = joined_predict.Visits

In [ ]:
max_log_y = np.max(np.log(joined_samp.Visits))
y_train = np.log(y_train_orig)/max_log_y
y_valid = np.log(y_valid_orig)/max_log_y
# I dont think we wan to do this... but im going to anyway
y_predict = np.log(y_predict_orig)/max_log_y

In [ ]:
#
map_train = split_cols(cat_map_train) + [contin_map_train]
map_valid = split_cols(cat_map_valid) + [contin_map_valid]
map_predict = split_cols(cat_map_predict) + [contin_map_predict]

In [ ]:
#
map_train = split_cols(cat_map_train) + split_cols(contin_map_train)
map_valid = split_cols(cat_map_valid) + split_cols(contin_map_valid)
map_predict = split_cols(cat_map_predict) + split_cols(contin_map_predict)

In [ ]:
cat_map_info(cat_map_fit.features[1])

In [ ]:
contin_inp = Input((contin_cols,), name='contin')
contin_out = Dense(contin_cols*10, activation='relu', name='contin_d')(contin_inp)
#contin_out = BatchNormalization()(contin_out)

## Save things into pickles
Everything after the pickles was very unstable so I needed to save all the information. 

In [ ]:
pickle.dump(map_train, open('map_train.pickle', 'wb'))
pickle.dump(y_train, open('y_train.pickle', 'wb'))

In [ ]:
pickle.dump(map_valid, open('map_valid.pickle', 'wb'))
pickle.dump(y_valid, open('y_valid.pickle', 'wb'))

In [ ]:
pickle.dump(map_predict, open('map_predict.pickle', 'wb'))
pickle.dump(y_predict, open('y_predict.pickle', 'wb'))

In [ ]:
pickle.dump(joined_train, open('joined_train.pickle', 'wb'))

In [ ]:
pickle.dump(joined_valid, open('joined_valid.pickle', 'wb'))
pickle.dump(joined_predict, open('joined_predict.pickle', 'wb'))

In [ ]:
pickle.dump(contin_cols, open('contin_cols.pickle', 'wb'))

In [ ]:
pickle.dump(max_log_y, open('max_log_y.pickle', 'wb'))
pickle.dump(y_train_orig, open('y_train_orig.pickle', 'wb'))

## Start Building Models

# ***Everything Below here was abandoned; Please look at Webtraffic-JustTraining-Final for the rest of the code.***

In [ ]:
import keras.backend as K

def customLoss(yTrue,yPred):
    return K.sum(K.log(yTrue) - K.log(yPred))

def smape1(y_pred, y_true):
    denominator = y_true + y_pred
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0
    return 200 * np.nanmean(diff, axis=0)

def smape(y_pred, y_true):
    K.denominator = y_true + y_pred
    K.diff = np.abs(y_true - y_pred) / K.denominator
    #K.diff[K.denominator == 0] = 0
    return 200 * (K.diff)

def smape_fast( y_pred, y_true ): # y_true= y_valid_orig
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

def smape_fast1( y_true, y_pred ): # y_true= y_valid_orig
    K.out = 0
    hold= len(y_valid_orig[0])
    print (y_true)
    for i in range(hold):
        K.a = y_true[i]
        K.b = y_pred[i]
        K.c = K.a+K.b
        if c == 0:
            continue
        K.out += int(K.math.fabs(K.a - K.b) / K.c)
    K.out *= (200.0 / y_true.shape[0])
    return K.out

In [ ]:
def sMAPE(y_true, y_pred):
    #Symmetric mean absolute percentage error
    #Should this be 200?
    return 200 * K.mean(K.abs(y_pred - y_true) / (K.abs(y_pred) + K.abs(y_true)), axis=-1)

def mean_signed_difference(y_true, y_pred):
    #True MSD does not have abs() but we need to always minimize this function
    return K.abs(K.sum((y_pred - y_true)/y_pred.ndim, axis=-1))

In [ ]:
embs = [get_emb(feat) for feat in cat_map_fit.features]
#conts = [get_contin(feat) for feat in contin_map_fit.features]
#contin_d = [d for inp,d in conts]
x = merge([emb for inp,emb in embs] + [contin_out], mode='concat')
#x = merge([emb for inp,emb in embs] + contin_d, mode='concat')

x = Dropout(0.04)(x)
x = Dense(1000, activation='relu', init='uniform')(x)
x = Dense(500, activation='relu', init='uniform')(x)
x = Dropout(0.4)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model([inp for inp,emb in embs] + [contin_inp], x)
#model = Model([inp for inp,emb in embs] + [inp for inp,d in conts], x)
#model.compile('adam', mse)
model.compile(loss=sMAPE, optimizer = 'adam')
#model.compile('adam', 'mean_absolute_error')
#model.compile(Adam(), 'mse')

### Start training


In [ ]:
model.lr=0.001

In [ ]:
RunningEpochs()

In [ ]:
%%time 
#Was nb_epoch=25
# Batch was 128
# 256 for 90 days is 6 mins
# 512 in for 90 days was 5 minutes5
# 1024 with 180 = 4 mins
# 4096 will run in 6 mins
hist = model.fit(map_train, y_train, batch_size=1028, nb_epoch=1, verbose=1, validation_data=(map_valid, y_valid))

In [ ]:
#model.save_weights(path+'webTraffic128x600.h5',overwrite=True)

In [ ]:
#model.history
#hist.history

In [ ]:
#plot_train(hist)

In [ ]:
# Makes the prediction
# Map_valid are the values fed into the machine
# preds = np.squeeze(model.predict(map_valid, 64))

# Start Predictions

In [ ]:
# Was 128
validpreds =model.predict(map_valid, 64, verbose=1)
listValidpreds=log_max_inv(validpreds,max_log_y)

In [ ]:
# Was 128
# 256 finishes in a minute
# 128 finishes in 2 minutes
Futurepreds=model.predict(map_predict, 64, verbose=1)

In [ ]:
#These are the values for submission
listFuturepreds=log_max_inv(Futurepreds,max_log_y)

In [ ]:
#Real Values
plt.plot(joined_valid.Visits, color='g')
# Valid set trained
plt.plot(listValidpreds, color='black')
# Future predictions
plt.plot(listFuturepreds, color='b')
plt.axhline(y=tables.Visits.mean(), color='r')
plt.show()

In [ ]:
ymean=y_train_orig.mean()
ystd=y_train_orig.std()

In [ ]:
len(listValidpreds)

In [ ]:
newsubm = joined_predict.assign(Visits=pd.Series(listFuturepreds.flatten().tolist()))
sepsubm = joined_valid.assign(Visits=pd.Series(listValidpreds.flatten().tolist()))

In [ ]:
display(DataFrameSummary(newsubm).summary())

In [ ]:
display(DataFrameSummary(tables).summary())

In [ ]:
import random
for i in range(5):
    testname=joined_train['Page'][random.randint(0, len(joined_train))]
    # plt.plot(listValidpreds, color='black') sepsubm
    submit1=newsubm.loc[newsubm['Page'] == testname]
    submit2=joined_train.loc[joined_train['Page'] == testname]
    submit3=joined_valid.loc[joined_valid['Page'] == testname]
    submit4=sepsubm.loc[sepsubm['Page'] == testname]
    #print(submit3)
    #plt.plot(submit1.Visits, color='b')
    if (SubmissionRun):#/==False):
        # Predicted amounts
        plt.plot(submit3.Visits, color='b')
        # Valid amounts 
        plt.plot(submit4.Visits, color='g')
    plt.axhline(submit2.Visits.mean(), color='r')
    print (testname)
    print (len(submit1),len(submit3))
    print (submit2.Visits.mean())    
    plt.show()

In [ ]:
allDone()

In [ ]:
newsubm

## Submit File
For when all the predictions and visits have been guessed.

In [ ]:
# Subm 
## ss_id are the ids; Combines page and date
## is the predicted visits for the page and date
subm2 = pd.DataFrame({'Id': newsubm.Id, 'Visits': newsubm.Visits})

In [ ]:
subm2[:5]

In [ ]:
# Makes the submission name out of folders
numFiles=len(os.listdir(path+'submits/'))
subm_name = 'submission'+str(numFiles)+'.csv.gz'
submission=subm2

In [ ]:
submission.to_csv((path+'submits/'+subm_name), index=False, compression='gzip')

In [ ]:
print ('http://localhost:8888/view/data/webTraffic/submits/'+subm_name)